# pandas

[`pandas`](http://pandas.pydata.org) is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

With `numpy` (and `scipy`), Python is a great platform for general purpose numerical computing providing much of the functionality of, say matlab.  What is lacking in the low level array routines provided by `numpy` provides are streamlined procedures for data analysis.  `pandas` provides several key data structures that make data analysis in Python feel natural.  `pandas` is built on `numpy` (indeed, nearly every Python library targeted at scientific computing is), but instead of arrays, `pandas` uses two slightly different "fundamental" data structures: the data `Series` and the `DataFrame`.


It is customary to import `pandas` as

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

(we've also imported `Series` and `DataFrame` directly since we'll be working with those frequently.)

## Creating a DataFrame

There are several methods of `DataFrame` creation.  For the following data,

| A | B | C | D |
|---|---|---|---|
|0.38|0.78|0.95|0.91|
|0.61|0.96|0.48|0.09|
|0.47|0.82|0.84|0.16|

a `DataFrame` can by creating by dictionary, whose keys form the `DataFrame`'s header and values the `DataFrame`'s columns:

In [ ]:
data = {"A": [0.38, 0.61, 0.47],
        "B": [0.78, 0.96, 0.82],
        "C": [0.95, 0.48, 0.84],
        "D": [0.91, 0.09, 0.16]}
df = DataFrame(data)
df

Alternatively, a `DataFrame` can be created from a nested sequence.  

In [ ]:
data = [[0.38, 0.78, 0.95, 0.91],
        [0.61, 0.96, 0.48, 0.09],
        [0.47, 0.82, 0.84, 0.16]]
df = DataFrame(data)
df

Default column names `0`, `1`, `2`, and `3` are used.  Alternatively, the `columns` keyword can by passed to the `DataFrame` constructor

In [ ]:
data = [[0.38, 0.78, 0.95, 0.91],
        [0.61, 0.96, 0.48, 0.09],
        [0.47, 0.82, 0.84, 0.16]]
df = DataFrame(data, columns=('A', 'B', 'C', 'D'))
df

The columns of the `DataFrame` need not be homogeneous in type

In [ ]:
df2 = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                    'D' : np.array([3] * 4,dtype='int32'),
                    'E' : pd.Categorical(["test","train","test","train"]),
                    'F' : 'foo' })
df2

Each `Series` in `df2` has a different data type

In [ ]:
df2.dtypes

Data can also be read directly from many file types.  Below, win, loss, and payroll data for major league baseball teams in 2001 are read from an excel spreadsheet (from http://powerreporting.com/files).  This data will be used for the remainder of this notebook.

In [ ]:
df = pd.read_excel('aux/baseball.xls', sheetname='2001 salary v performance', index_col=0)
df

Passing `index_col=0` instructed `read_excel` to treat the first column as the `DataFrame` index, otherwise, the index would have been set to $0-n$, where $n$ is the number of data rows found in the excel table.

Tab completion for column names (as well as public attributes) is automatically enabled. In the cell below, type `df.` and [Tab]:

In [ ]:
df.

The column names `Team`, `Won`, `Loss`, and `Payroll` show up in the tab completion, as do the other methods of the `DataFrame` object.

## Viewing data

See the top and bottom rows of the `DataFrame`

In [ ]:
df.head()

In [ ]:
df.tail(3)

Display the index, columns, and the underlying numpy data

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

Show a quick statistical summary of the data

In [ ]:
df.describe()

`describe` is smart enough to operate only on numerical data

Sort by an axis

In [ ]:
df.sort_index(axis=1, ascending=False)

Sorting by values

In [ ]:
df.sort(columns='Payroll')

Not surprisingly, the Boston Red Sox and New York Yankes had the highest payrolls.

## Data selection

Selecting a single column, which yields a `Series`, equivalent to `df.Won`

In [ ]:
df['Won']

Selecting via [], which slices the rows.

In [ ]:
df[0:3]

### Selection by label

For getting a cross section using a label

In [ ]:
df.loc['Anaheim Angels', ['Won', 'Loss']]

Selecting on a multi-axis by label

In [ ]:
df.loc[:, ['Won', 'Loss']]

Showing label slicing, both endpoints are included

In [ ]:
df.loc['Anaheim Angels':'Chicago Cubs', ['Won']]

Reduction in the dimensions of the returned object

In [ ]:
df.loc['Baltimore Orioles', ['Won', 'Loss']]

For getting fast access to a scalar

In [ ]:
df.at['Atlanta Braves', 'Won']

### Selection by position

Select via the position of the passed integers

In [ ]:
df.iloc[3]

By integer slices, acting similar to numpy/python

In [ ]:
df.iloc[1:5, 0:2]

By lists of integer position locations, similar to the numpy/python style

In [ ]:
df.iloc[[0, 2, 9], [0, 3]]

For slicing rows explicitly

In [ ]:
df.iloc[1:3, :]

For slicing columns explicitly

In [ ]:
df.iloc[:, 1:3]

For getting a value explicitly

In [ ]:
df.iloc[5,1]

For getting fast access to a scalar (equiv to the prior method)

In [ ]:
df.iat[5,1]

### Boolean indexing

Using a single column’s values to select data.

In [ ]:
df[df.Payroll > 100000000]

(apparently, $100,000,000 can't always buy you a playoff spot!)

Boolean indexing supports multiple conditions using the elementwise and `&` and or `|` operators:

In [ ]:
df[(df.Payroll < 100000000) & (df.Playoff == 'Yes')]

The `isin()` method can be used for filtering.  Here, the history of World Series winners from 1903 is read in and we filter teams based on whether or not they have won a world series

In [ ]:
df2 = pd.read_excel('aux/baseball.xls', sheetname='world series history', index_col=0)
df[df.index.isin(df2['World Series Champ'])]

The `~` operator is the bitwise `not` operator:

In [ ]:
df[~df.index.isin(df2['World Series Champ'])]

### Setting

Setting by assigning with a numpy array

Here, the `Playoff` column is converted to a boolean from the given 'Yes' and 'No' strings.

In [ ]:
df['Playoff'] = np.where(df['Playoff'] == 'No', False, True)
df

In [ ]:
df.dtypes

In [ ]:
df['Win'] = df['Won'] / (df['Won'] + df['Loss'])
df['Cost per win'] = df['Payroll'] / df['Won']

Setting a new column automatically aligns the data by the indexes

In [ ]:
df['Payroll rank'] = df['Payroll'].rank(ascending=False)
df['Win rank'] = df['Win'].rank(ascending=False)
df['Cost rank'] = df['Cost per win'].rank(ascending=False)
df

## Operations

### Statistics

Operations in general exclude missing data.

Performing a descriptive statistic

In [ ]:
df[['Payroll', 'Cost per win']].mean()

Operations can be performed on the other axis

In [ ]:
df.mean(1)

Though, in this example the meaning of the result is unclear.

## Grouping

By “group by”, `pandas` is referring to a process involving one or more of the following steps

- Splitting the data into groups based on some criteria
- Applying a function to each group independently
- Combining the results into a data structure

Grouping and then applying a function sum to the resulting groups.

In [ ]:
df.groupby('Playoff').mean()

In [ ]:
def func(x):
    return x['Payroll'].mean()
df.groupby('Playoff').apply(func)

## Writing data to a file

### csv

In [ ]:
df.to_csv('aux/foo.csv')

In [ ]:
df1 = pd.read_csv('aux/foo.csv', index_col=0)
df1

### Excel

In [ ]:
df.to_excel('aux/foo.xlsx', sheet_name='Sheet1', index_label='Team')

In [ ]:
df1 = pd.read_excel('aux/foo.xlsx', 'Sheet1')
df1

The `to_excel()` method does not seem to handle the index properly so that when the sheet is read it is not consistent with the original `DataFrame`.  The `to_csv()` method seems to handle it correctly.